# Model testing
Try different models to see which perform best.

## 0. Import required packages

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from scipy.stats import randint as sp_randint
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostRegressor
from lmfit import Model
from xgboost import XGBRegressor
from collections import OrderedDict
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import neighbors
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import GenericUnivariateSelect
from sklearn.feature_selection import f_regression
from scipy.stats import spearmanr
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

pl.style.use('seaborn')
pl.rc('font',family='Arial')

## 1. Read in and shape data

In [2]:
train_data = pd.read_csv("SubCh1_TrainingData.csv")

# train data:
train_data['Timepoint'] = [1 if i == '24HR' else 0 for i in train_data['Timepoint']]
train_data['Treatment'] = [1 if i == 'DHA' else 0 for i in train_data['Treatment']]
train_data['BioRep'] = [int(i[-1]) for i in train_data['BioRep']]

# generate list of gene names:
genes = list(train_data.drop(['DHA_IC50','Sample_Name','Isolate','Timepoint','Treatment','BioRep'], axis=1).columns)

In [5]:
# split train data:
X_A = train_data[(train_data['Timepoint'] == 0)&(train_data['Treatment'] == 0)].drop(['DHA_IC50','Sample_Name','Isolate','Timepoint','Treatment','BioRep'], axis = 1)
X_B = train_data[(train_data['Timepoint'] == 1)&(train_data['Treatment'] == 0)].drop(['DHA_IC50','Sample_Name','Isolate','Timepoint','Treatment','BioRep'], axis = 1)
X_C = train_data[(train_data['Timepoint'] == 0)&(train_data['Treatment'] == 1)].drop(['DHA_IC50','Sample_Name','Isolate','Timepoint','Treatment','BioRep'], axis = 1)
X_D = train_data[(train_data['Timepoint'] == 1)&(train_data['Treatment'] == 1)].drop(['DHA_IC50','Sample_Name','Isolate','Timepoint','Treatment','BioRep'], axis = 1)

# pull out target feature:
Y = train_data[(train_data['Timepoint'] == 0)&(train_data['Treatment'] == 0)]['DHA_IC50']

# extract biorep feature:
bioreps = train_data[(train_data['Timepoint'] == 0)&(train_data['Treatment'] == 0)]['BioRep']

# rename train data columns:
X_A.columns = ['A' + str(i) for i in range(1,5541)]
X_B.columns = ['B' + str(i) for i in range(1,5541)]
X_C.columns = ['C' + str(i) for i in range(1,5541)]
X_D.columns = ['D' + str(i) for i in range(1,5541)]

# reset indices
X_A = X_A.reset_index().drop(['index'],axis=1)
X_B = X_B.reset_index().drop(['index'],axis=1)
X_C = X_C.reset_index().drop(['index'],axis=1)
X_D = X_D.reset_index().drop(['index'],axis=1)

# combine data frames:
X = pd.concat([X_A,X_B,X_C,X_D], axis=1)
X.head(10)

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,D5531,D5532,D5533,D5534,D5535,D5536,D5537,D5538,D5539,D5540
0,1.311350,-1.613464,-1.298663,-1.441343,-1.735923,0.173112,2.466366,1.383979,-0.115130,0.287468,...,0.210607,-0.540993,-3.674097,-1.652979,-2.255490,-4.554757,-0.381422,-1.415857,-4.121011,-2.486528
1,0.997722,-1.553994,-1.960600,-1.424590,0.023609,0.420973,1.128427,0.722659,1.878123,-0.065159,...,-0.447109,0.450649,-4.464408,-0.977954,-2.012559,-4.538550,-2.333890,-2.342496,-4.774197,-1.794568
2,0.389508,-2.139782,-0.584985,-1.085373,0.803247,0.766617,1.701689,0.926101,1.600687,0.435633,...,-0.070151,0.024133,-2.215227,-1.957654,-2.188635,-4.424748,-2.986927,-1.722201,-3.995680,-0.902979
3,0.348560,-1.562540,-0.586732,-0.834661,1.096979,0.121817,1.623373,-0.654405,0.221121,0.998601,...,-1.288305,0.806314,-3.733712,-1.990368,-1.633418,-5.533077,-3.283316,-2.104227,-5.767710,-2.177930
4,0.138276,-1.612280,-1.362990,-1.360318,0.488124,0.365410,0.739845,-0.654702,2.170263,0.630418,...,0.279816,0.115002,-1.305902,-0.679212,-2.099512,-5.955507,-0.920594,-1.626372,-4.422711,-1.408485
5,-0.332565,-1.280348,-0.817751,-0.480521,-0.098970,0.112246,0.773993,-0.969944,1.117705,0.872166,...,0.375290,1.113241,-4.016287,-1.327287,-2.375500,-4.379304,-2.752906,-1.939162,-3.500963,-0.796143
6,-1.140942,-0.460872,0.588564,-1.214647,1.210955,1.336895,-0.857090,-1.884711,-0.001943,1.320737,...,0.666558,-2.016912,-4.682891,-0.254524,-2.668047,-6.573239,-3.264155,-0.671344,-5.312867,-2.248219
7,0.426584,-1.118851,-0.938263,-1.442067,0.747063,0.268388,1.062841,0.494635,0.866481,0.501923,...,-1.793459,-0.116719,-4.535173,-0.537190,-2.472669,-6.261742,-3.134708,-1.969545,-5.157691,-2.637917
8,0.037085,-1.488644,-1.838770,-1.066493,-1.305085,0.059484,2.128986,-0.527511,1.616871,0.235093,...,-0.478144,0.125283,-5.360958,-2.386122,-1.886257,-6.127057,-3.231487,-2.349583,-4.977880,-1.047555
9,-0.037537,-1.495531,-1.801568,-0.526676,-1.831555,0.494413,1.286978,-1.589983,1.060466,0.836018,...,1.213397,-1.032989,-8.298501,-0.080207,-2.335635,-4.606785,-2.849864,-2.028372,-3.179649,-2.205214


## 2. Feature engineering

In [6]:
def drop_univ(X_orig, Y, features):
    X = StandardScaler().fit_transform(X_orig)
    X_uni = GenericUnivariateSelect(f_regression,'k_best', param=features).fit_transform(X, Y.values.ravel())
    X_uni = pd.DataFrame(data = X_uni)

    return X_uni

In [7]:
# scale and transform data
X_s = StandardScaler().fit_transform(X)
transformer = GenericUnivariateSelect(f_regression,'k_best', param=70)
X_uni = transformer.fit_transform(X_s, Y.values.ravel())
X_uni = pd.DataFrame(data = X_uni)

## 3. Model testing

### 3.1. Define metric and split data

In [8]:
def spear_r(y_true, y_pred):
    return spearmanr(y_true, y_pred)[0]

In [9]:
X_train, X_test , Y_train , Y_test = train_test_split(X_uni,Y,test_size=0.33,random_state=10)

### 3.2. Linear Regression

In [53]:
linear_model = LinearRegression()
linear_model.fit(X_train, Y_train)
Y_pred = linear_model.predict(X_test)

linear_model_mse = mean_squared_error(Y_test, Y_pred)
linear_model_spear = spearmanr(Y_test,Y_pred)[0]
linear_model_cv = np.mean(cross_val_score(linear_model, X_train, Y_train.values.ravel(), cv=5))
linear_model_cvspear = np.mean(cross_val_score(linear_model, X_train, Y_train.values.ravel(), cv=5,scoring=make_scorer(spear_r)))

print('mse: ' + str(mean_squared_error(Y_test, Y_pred)))
print('accuracy: ' + str(linear_model.score(X_test,Y_test)))
print('spearman: ' + str(spearmanr(Y_test,Y_pred)[0]))
print('cv score: ' + str(linear_model_cv))
print('cv spear: ' + str(linear_model_cvspear))

mse: 0.31850268822322364
accuracy: -0.24787444027216177
spearman: 0.47694883162960167
cv score: -0.10997663308315149
cv spear: 0.42408328160562


### 3.3. SVR

In [52]:
svr_model = SVR()
svr_model.fit(X_train, Y_train.values.ravel())
Y_pred = svr_model.predict(X_test)

svr_model_mse = mean_squared_error(Y_test, Y_pred)
svr_model_spear = spearmanr(Y_test,Y_pred)[0]
svr_model_cv = np.mean(cross_val_score(svr_model, X_train, Y_train.values.ravel(), cv=5))
svr_model_cvspear = np.mean(cross_val_score(svr_model, X_train, Y_train.values.ravel(), cv=5,scoring=make_scorer(spear_r)))

print('mse: ' + str(mean_squared_error(Y_test, Y_pred)))
print('accuracy: ' + str(svr_model.score(X_test,Y_test)))
print('spearman: ' + str(spearmanr(Y_test,Y_pred)[0]))
print('cv score: ' + str(svr_model_cv))
print('cv spear: ' + str(svr_model_cvspear))

mse: 0.14672944368566893
accuracy: 0.42512283512541765
spearman: 0.5111582592620783
cv score: 0.3490669215921106
cv spear: 0.5829288774372416


### 3.4. KNN

In [51]:
knn_model = neighbors.KNeighborsRegressor(5,weights='distance')
knn_model.fit(X_train, Y_train.values.ravel())
Y_pred = knn_model.predict(X_test)

knn_model_mse = mean_squared_error(Y_test, Y_pred)
knn_model_spear = spearmanr(Y_test,Y_pred)[0]
knn_model_cv = np.mean(cross_val_score(knn_model, X_train, Y_train.values.ravel(), cv=5))
knn_model_cvspear = np.mean(cross_val_score(knn_model, X_train, Y_train.values.ravel(), cv=5,scoring=make_scorer(spear_r)))

print('mse: ' + str(mean_squared_error(Y_test, Y_pred)))
print('accuracy: ' + str(knn_model.score(X_test,Y_test)))
print('spearman: ' + str(spearmanr(Y_test,Y_pred)[0]))
print('cv score: ' + str(knn_model_cv))
print('cv spear: ' + str(knn_model_cvspear))

mse: 0.17207615458826678
accuracy: 0.3258159411812387
spearman: 0.3673795054444229
cv score: 0.3541218783735642
cv spear: 0.6096802883668038


### 3.5. Decision Tree

In [50]:
tree_model = DecisionTreeRegressor(max_depth=50)
tree_model.fit(X_train, Y_train)
Y_pred = tree_model.predict(X_test)

tree_model_mse = mean_squared_error(Y_test, Y_pred)
tree_model_spear = spearmanr(Y_test,Y_pred)[0]
tree_model_cv = np.mean(cross_val_score(tree_model, X_train, Y_train.values.ravel(), cv=5))
tree_model_cvspear = np.mean(cross_val_score(tree_model, X_train, Y_train.values.ravel(), cv=5,scoring=make_scorer(spear_r)))

print('mse: ' + str(mean_squared_error(Y_test, Y_pred)))
print('accuracy: ' + str(tree_model.score(X_test,Y_test)))
print('spearman: ' + str(spearmanr(Y_test,Y_pred)[0]))
print('cv score: ' + str(tree_model_cv))
print('cv spear: ' + str(tree_model_cvspear))

mse: 0.6720055208695652
accuracy: -1.6328773483606862
spearman: 0.022376929668638877
cv score: 0.0279993467107968
cv spear: 0.38097910624415154


### 3.6. Random Forest

In [49]:
np.random.seed(999)
forest_model = RandomForestRegressor(max_features=10,n_estimators=100, bootstrap=False,random_state = 999)
forest_model.fit(X_train, Y_train.values.ravel())
Y_pred = forest_model.predict(X_test)

forest_model_mse = mean_squared_error(Y_test, Y_pred)
forest_model_spear = spearmanr(Y_test,Y_pred)[0]
forest_model_cv = np.mean(cross_val_score(forest_model, X_train, Y_train.values.ravel(), cv=5))
forest_model_cvspear = np.mean(cross_val_score(forest_model, X_train, Y_train.values.ravel(), cv=5,scoring=make_scorer(spear_r)))

print('mse: ' + str(mean_squared_error(Y_test, Y_pred)))
print('accuracy: ' + str(forest_model.score(X_test,Y_test)))
print('spearman: ' + str(spearmanr(Y_test,Y_pred)[0]))
print('cv score: ' + str(forest_model_cv))
print('cv spear: ' + str(forest_model_cvspear))

mse: 0.22180673402695653
accuracy: 0.1309745119686473
spearman: 0.3371363882620885
cv score: 0.318037411209186
cv spear: 0.5146032414169597


### 3.7. XGBoost

In [48]:
xgb_model = XGBRegressor(colsample_bytree=0.4, gamma=0, learning_rate=0.07, max_depth=3, min_child_weight=1.5,\
                 n_estimators=10000, reg_alpha=0.75, reg_lambda=0.45, subsample=0.6, seed=42)
xgb_model.fit(X_train, Y_train.values.ravel())
Y_pred = xgb_model.predict(X_test)

xgb_model_mse = mean_squared_error(Y_test, Y_pred)
xgb_model_spear = spearmanr(Y_test,Y_pred)[0]
xgb_model_cv = np.mean(cross_val_score(xgb_model, X_train, Y_train.values.ravel(), cv=5))
xgb_model_cvspear = np.mean(cross_val_score(xgb_model, X_train, Y_train.values.ravel(), cv=5,scoring=make_scorer(spear_r)))

print('mse: ' + str(mean_squared_error(Y_test, Y_pred)))
print('accuracy: ' + str(xgb_model.score(X_test,Y_test)))
print('spearman: ' + str(spearmanr(Y_test,Y_pred)[0]))
print('cv score: ' + str(xgb_model_cv))
print('cv spear: ' + str(xgb_model_cvspear))

mse: 0.21787572260082727
accuracy: 0.14637597909738487
spearman: 0.3212711464615196
cv score: 0.29163697492825086
cv spear: 0.6317686375066041


### 3.8. AdaBoost

In [47]:
ada_model = AdaBoostRegressor(random_state=0, n_estimators=200)
ada_model.fit(X_train, Y_train.values.ravel())
Y_pred = ada_model.predict(X_test)

ada_model_mse = mean_squared_error(Y_test, Y_pred)
ada_model_spear = spearmanr(Y_test,Y_pred)[0]
ada_model_cv = np.mean(cross_val_score(ada_model, X_train, Y_train.values.ravel(), cv=5))
ada_model_cvspear = np.mean(cross_val_score(ada_model, X_train, Y_train.values.ravel(), cv=5,scoring=make_scorer(spear_r)))

print('mse: ' + str(mean_squared_error(Y_test, Y_pred)))
print('accuracy: ' + str(ada_model.score(X_test,Y_test)))
print('spearman: ' + str(spearmanr(Y_test,Y_pred)[0]))
print('cv score: ' + str(ada_model_cv))
print('cv spear: ' + str(ada_model_cvspear))

mse: 0.20688111683807886
accuracy: 0.1894521854199737
spearman: 0.24969135422340685
cv score: 0.24450024937428655
cv spear: 0.4128903022267096


### 3.9. Gradient Boost

In [46]:
gb_model = GradientBoostingRegressor(loss='quantile', n_estimators=50)
gb_model.fit(X_train, Y_train.values.ravel())
Y_pred = gb_model.predict(X_test)

gb_model_mse = mean_squared_error(Y_test, Y_pred)
gb_model_spear = spearmanr(Y_test,Y_pred)[0]
gb_model_cv = np.mean(cross_val_score(gb_model, X_train, Y_train.values.ravel(), cv=5))
gb_model_cvspear = np.mean(cross_val_score(gb_model, X_train, Y_train.values.ravel(), cv=5,scoring=make_scorer(spear_r)))

print('mse: ' + str(mean_squared_error(Y_test, Y_pred)))
print('accuracy: ' + str(gb_model.score(X_test,Y_test)))
print('spearman: ' + str(spearmanr(Y_test,Y_pred)[0]))
print('cv score: ' + str(gb_model_cv))
print('cv spear: ' + str(gb_model_cvspear))

mse: 0.25973560969409804
accuracy: -0.017628549303208896
spearman: 0.43861850695954263
cv score: -0.9489951935741144
cv spear: 0.34554517395172907


## 4. Model comparison

In [54]:
models = pd.DataFrame([['Linear', linear_model.score(X_test,Y_test),linear_model_spear,linear_model_cv,linear_model_cvspear,linear_model_mse], \
                      ['SVR', svr_model.score(X_test,Y_test),svr_model_spear,svr_model_cv,svr_model_cvspear,svr_model_mse], \
                      ['K Nearest Neighbors', knn_model.score(X_test,Y_test),knn_model_spear,knn_model_cv,knn_model_cvspear,knn_model_mse], \
                      ['Decision Tree',tree_model.score(X_test,Y_test),tree_model_spear,tree_model_cv,tree_model_cvspear,tree_model_mse], \
                      ['Random Forest', forest_model.score(X_test,Y_test),forest_model_spear,forest_model_cv,forest_model_cvspear,forest_model_mse], \
                      ['XGBoost', xgb_model.score(X_test,Y_test),xgb_model_spear,xgb_model_cv,xgb_model_cvspear,xgb_model_mse], \
                      ['AdaBoost', ada_model.score(X_test,Y_test),ada_model_spear,ada_model_cv,ada_model_cvspear,ada_model_mse], \
                      ['Gradient Boost', gb_model.score(X_test,Y_test),gb_model_spear,gb_model_cv,gb_model_cvspear,gb_model_mse]])
models.columns = ['Model', 'Test Score','Spearman', 'CV score', 'Spearman CV', 'MSE']

In [55]:
models

,Model,Test Score,Spearman,CV score,Spearman CV,MSE
0,Linear,-0.247874,0.476949,-0.109977,0.424083,0.318503
1,SVR,0.425123,0.511158,0.349067,0.582929,0.146729
2,K Nearest Neighbors,0.325816,0.367380,0.354122,0.609680,0.172076
3,Decision Tree,-1.632877,0.022377,0.027999,0.380979,0.672006
4,Random Forest,0.130975,0.337136,0.318037,0.514603,0.221807
5,XGBoost,0.146376,0.321271,0.291637,0.631769,0.217876
6,AdaBoost,0.189452,0.249691,0.244500,0.412890,0.206881
7,Gradient Boost,-0.017629,0.438619,-0.948995,0.345545,0.259736


Conclusion: SVR performs best for Test Score, Spearman, and MSE. XGBoost performs best for Spearman CV.